# EfficientNet — czym jest i główna idea

**EfficientNet** to rodzina efektywnych sieci konwolucyjnych (CNN) zaproponowana przez **Mingxing Tan i Quoc V. Le (Google, 2019)**.  
Została stworzona, aby osiągać wysoką dokładność w zadaniach wizji komputerowej przy **znacznie mniejszych kosztach obliczeniowych** niż wcześniejsze modele (ResNet, DenseNet, Inception).

---

## 🔹 Czym jest?
- Bazowy model **EfficientNet-B0** został znaleziony za pomocą **Neural Architecture Search (NAS)**.  
- Kolejne modele (**B1–B7**) powstają poprzez **skalowanie** bazowej architektury.  
- W konstrukcji wykorzystano:
  - bloki **MBConv** (*Mobile Inverted Bottleneck Convolution*),
  - mechanizm uwagi **Squeeze-and-Excitation (SE)**.

---

## 🔹 Główna idea — *Compound Scaling*
Zamiast powiększać tylko jeden wymiar sieci (np. głębokość), EfficientNet skaluje **trzy jednocześnie**:

- **depth** → głębokość (liczba warstw),
- **width** → szerokość (liczba kanałów w warstwach),
- **resolution** → rozdzielczość obrazu wejściowego.

Skalowanie odbywa się według wzoru:

depth       = α^φ <br>
width       = β^φ <br>
resolution  = γ^φ <br>


gdzie:
- `φ` — współczynnik skali (kontroluje wielkość modelu),  
- `α, β, γ` — stałe dobrane empirycznie, aby uzyskać najlepszy balans.

---

## 🔹 Zalety
- ✅ Bardzo wysoka dokładność przy małej liczbie parametrów.  
- ✅ Możliwość wyboru modelu B0–B7 w zależności od dostępnych zasobów.  
- ✅ Dobre do transfer learningu w klasyfikacji obrazów i medycynie.  

---


In [6]:
import os
import shutil
from pathlib import Path
import kaggle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
import random
from torchvision.models import efficientnet_b0
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
import cv2
from PIL import Image



In [8]:
def load(tr_path):
    classes, class_paths = zip(*[(label, os.path.join(tr_path, label, image))
                                 for label in os.listdir(tr_path) if os.path.isdir(os.path.join(tr_path, label))
                                 for image in os.listdir(os.path.join(tr_path, label))])

    return pd.DataFrame({'Class Path': class_paths, 'Class': classes})

SEED = 900729
dir_path = './data'
train_path = './data/Training'
test_path = './data/Testing'
validation_path = './data/Validation'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Devic: {device}")

if os.path.exists(dir_path):
    print("Dane juz istnieją")
else:
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('masoudnickparvar/brain-tumor-mri-dataset', path='./data', unzip=True)
    tr_orginal_df = load(train_path)
    ts_df = load(test_path)
    val_df, tr_df = train_test_split(tr_orginal_df, train_size=0.25, random_state=SEED, stratify=tr_orginal_df['Class'])
    validation_path = './data/Validation'
    Path(validation_path).mkdir(exist_ok=True)

    classes = ['glioma', 'meningioma', 'notumor', 'pituitary']
    for class_name in classes:
        class_validation_path = Path(validation_path) / class_name
        class_validation_path.mkdir(exist_ok=True)

    errors = []

    for idx, row in val_df.iterrows():
        try:
            source_path = Path(row['Class Path'])

            filename = source_path.name
            class_name = row['Class']
            destination_path = Path(validation_path) / class_name / filename

            shutil.move(str(source_path), str(destination_path))

        except Exception as e:
            errors.append({
                'file': row['Class Path'],
                'class': row['Class'],
                'error': str(e)
            })


    if errors:
        print(f"Errors: {len(errors)}")
        for error in errors:
            print(f"  - {error['class']}: {Path(error['file']).name} - {error['error']}")

    val_df_updated = val_df.copy()
    val_df_updated['Class Path'] = val_df_updated.apply(
        lambda row: str(Path(validation_path) / row['Class'] / Path(row['Class Path']).name),
        axis=1
    )

    tr_df_updated = load(train_path)

    val_df = val_df_updated
    tr_df = tr_df_updated
    print("Pomyślnie pobrano dane")



data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_path, transform=data_transform)
test_dataset  = datasets.ImageFolder(root=test_path, transform=data_transform)
val_dataset   = datasets.ImageFolder(root=validation_path, transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)


Devic: cpu
Dane juz istnieją


In [10]:
model = efficientnet_b0(pretrained=True)
num_classes = len(train_dataset.classes)

in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)



/Users/fidok/Developer/WakacyjneWyzwanieSOLVRO-ML/FinalProject/Brain_Tumor_Team_Projekt4/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fidok/Developer/WakacyjneWyzwanieSOLVRO-ML/FinalProject/Brain_Tumor_Team_Projekt4/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
EPOCHS = 10
best_loss = float("inf")

if os.path.exists('best_model.pth'):
    model.load_state_dict(torch.load('best_model.pth'))
    print("Wczytano zapisany model")
else:
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.4f}")
        
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model.state_dict(), "best_model.pth")
            print("Zapisano nowy najlepszy model")


Epoch 1/10, Loss: 0.0913
Zapisano nowy najlepszy model
Epoch 2/10, Loss: 0.0618
Zapisano nowy najlepszy model
Epoch 3/10, Loss: 0.0521
Zapisano nowy najlepszy model
Epoch 4/10, Loss: 0.0400
Zapisano nowy najlepszy model
Epoch 5/10, Loss: 0.0422
Epoch 6/10, Loss: 0.0381
Zapisano nowy najlepszy model


In [ ]:
best_loss = float('inf')
model.eval()
val_loss = 0.0
with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

val_loss /= len(val_loader)
if val_loss < best_loss:
    best_loss = val_loss
    counter = 0
    torch.save(model.state_dict(), 'best_model.pth')